# Pymagicc demo

See https://github.com/openclimatedata/pymagicc

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pymagicc

from ipywidgets import widgets
from IPython.display import Javascript

from functools import lru_cache

plt.style.use("ggplot")

In [2]:
@lru_cache(maxsize=32)
def run_pymagicc(scenario, core_climatesensitivity=3):
    return pymagicc.run(pymagicc.scenarios[scenario], core_climatesensitivity=core_climatesensitivity)

In [3]:
# Reference period
start = 1850
end = 1900

widget_scenario = widgets.Dropdown(
    title="Emissions Scenario", 
    value="RCP3PD",
    options=["RCP3PD", "RCP45", "RCP6", "RCP85"]
)

gases = pymagicc.rcp3pd["WORLD"].columns.values

widget_gas_1 = widgets.Dropdown(
    title="Emissions from", 
    value="FossilCO2",
    options=gases
)
widget_gas_2 = widgets.Dropdown(
    title="Emissions from", 
    value="OtherCO2",
    options=gases
)
widget_gas_3 = widgets.Dropdown(
    title="Emissions from", 
    value="CH4",
    options=gases
)
widget_gas_4 = widgets.Dropdown(
    title="Emissions from", 
    value="N2O",
    options=gases
)

style = {'description_width': 'initial'}
widget_core_climatesensitivity = widgets.FloatSlider(
    value=3, min=1.5, max=4.5, step=0.1, continuous_update=False,
    description="Climate Sensitivity",
    style=style
)

scen = pymagicc.scenarios["RCP3PD"]
data = scen["WORLD"]["FossilCO2"].loc[:2100]

widget_output = widgets.Output()

ui = widgets.VBox([
    widget_scenario, 
    widgets.HBox([widget_gas_1, widget_gas_2, widget_gas_3, widget_gas_4]),
    widget_core_climatesensitivity,
    widget_output])

def update(scenario, gas_1, gas_2, gas_3, gas_4, core_climatesensitivity):
    scen = pymagicc.scenarios[scenario]
    data = scen["WORLD"].loc[:2100]
    data.index.name = ""
    results = run_pymagicc(scenario, core_climatesensitivity=core_climatesensitivity)
    
    fig, axes = plt.subplots(2, 4, figsize=(16,10))
    data[gas_1].plot(ax=axes[0, 0])
    axes[0, 0].set_title(gas_1 + " [" + pymagicc.units[gas_1] + "]")
    data[gas_2].plot(ax=axes[0, 1])
    axes[0, 1].set_title(gas_2  + " [" + pymagicc.units[gas_2] + "]")
    data[gas_3].plot(ax=axes[0, 2])
    axes[0, 2].set_title(gas_3 + " [" + pymagicc.units[gas_3] + "]")
    data[gas_4].plot(ax=axes[0, 3])
    axes[0, 3].set_title(gas_4 + " [" + pymagicc.units[gas_4] + "]")
    
    for key in results.keys():
        results[key].index.name = ""
    temp = results["SURFACE_TEMP"].GLOBAL
    temp = temp - temp.loc[start:end].mean()
    temp.plot(ax=axes[1, 0])
    axes[1, 0].set_title("Surface Temperature [°C]")
    
    results["CO2_CONC"].GLOBAL.plot(ax=axes[1, 1])
    axes[1, 1].set_title("CO$_2$ Concentration [ppm]")
    
    results["GHG_RF"].GLOBAL.plot(ax=axes[1, 2])
    axes[1, 2].set_title("Radiative Forcing [W/m$^2$]")
    
    axes[1, 3].remove()

out = widgets.interactive_output(
    update, 
    {
        "scenario": widget_scenario, 
        "gas_1": widget_gas_1,
        "gas_2": widget_gas_2,
        "gas_3": widget_gas_3,
        "gas_4": widget_gas_4,
        "core_climatesensitivity": widget_core_climatesensitivity
    }
)
    
display(ui, out)

VBox(children=(Dropdown(options=('RCP3PD', 'RCP45', 'RCP6', 'RCP85'), value='RCP3PD'), HBox(children=(Dropdown(options=('FossilCO2', 'OtherCO2', 'CH4', 'N2O', 'SOx', 'CO', 'NMVOC', 'NOx', 'BC', 'OC', 'NH3', 'CF4', 'C2F6', 'C6F14', 'HFC23', 'HFC32', 'HFC43-10', 'HFC125', 'HFC134a', 'HFC143a', 'HFC227ea', 'HFC245fa', 'SF6'), value='FossilCO2'), Dropdown(index=1, options=('FossilCO2', 'OtherCO2', 'CH4', 'N2O', 'SOx', 'CO', 'NMVOC', 'NOx', 'BC', 'OC', 'NH3', 'CF4', 'C2F6', 'C6F14', 'HFC23', 'HFC32', 'HFC43-10', 'HFC125', 'HFC134a', 'HFC143a', 'HFC227ea', 'HFC245fa', 'SF6'), value='OtherCO2'), Dropdown(index=2, options=('FossilCO2', 'OtherCO2', 'CH4', 'N2O', 'SOx', 'CO', 'NMVOC', 'NOx', 'BC', 'OC', 'NH3', 'CF4', 'C2F6', 'C6F14', 'HFC23', 'HFC32', 'HFC43-10', 'HFC125', 'HFC134a', 'HFC143a', 'HFC227ea', 'HFC245fa', 'SF6'), value='CH4'), Dropdown(index=3, options=('FossilCO2', 'OtherCO2', 'CH4', 'N2O', 'SOx', 'CO', 'NMVOC', 'NOx', 'BC', 'OC', 'NH3', 'CF4', 'C2F6', 'C6F14', 'HFC23', 'HFC32', 'HFC43-10', 'HFC125', 'HFC134a', 'HFC143a', 'HFC227ea', 'HFC245fa', 'SF6'), value='N2O'))), FloatSlider(value=3.0, continuous_update=False, description='Climate Sensitivity', max=4.5, min=1.5, style=SliderStyle(description_width='initial')), Output()))

Output()

Surface temperature is relative to the pre-industrial 1850-1900 period.